## MAGMA experimnetations

In [1]:
import numpy as np
import sys

sys.path.append("../src/")
from kernels import *
#from MAGMA2 import MAGMA

Draw a random working grid $𝐭 \subset [0,10]$ of $N = 200$ timestamps, and a number $M$ of individuals.

In [2]:
t0 = 0
tN = 10 + 1e-3
N = 200
M = 3

t = np.linspace(t0, tN, N)
print(t.shape)

(200,)


Define a prior mean function : $m0(t) = at+b$, $\forall t ∈ 𝐭$ , where $a \in [−2,2]$ and $b \in [0,10]$ are drawn uniformly.

In [3]:
a = np.random.uniform(-2, 2)
b = np.random.uniform(0, 10)

m0 = a * t + b
print(m0.shape)

(200,)


Draw hyper-parameters uniformly for $\mu_0$’s kernel : $\theta_0 = \{v_0, l_0\}$, where $v_0 \in [1, exp(5)]$ and $l_0 \in [1, exp(2)]$.

In [4]:
v0 = np.random.uniform(1, np.exp(5))
l0 = np.random.uniform(1, np.exp(2))

theta0 = np.array([v0, l0])
print(theta0)

[8.25757947 5.96526702]


Draw $\mu_0(𝐭)  \sim N (m_0(𝐭),𝐊^𝐭_{\theta_0})$.

In [5]:
K_theta_0 = ExponentiatedQuadraticKernel.compute_all(theta0, t)
print(K_theta_0.shape)

(200, 200)


$\forall i \in I$, draw  $v_i \in [1, exp(5)]$, $l_i \in [1, exp(2)]$ and $\sigma^2_i \in [0, 1]$ uniformly.

In [6]:
v = np.random.uniform(1, np.exp(5), size=M)
l = np.random.uniform(1, np.exp(2), size=M)

Theta = np.c_[v, l]
Sigma = np.random.uniform(0, 1, size=M)

print(Theta)
print(Sigma)

[[79.30197058  1.19940676]
 [29.30172496  2.46409552]
 [ 7.18654032  5.44566582]]
[0.53973738 0.84107679 0.08771977]


$\forall i \in I$, draw a subset $t_i \subset t$ of $N_i = 30$ timestamps uniformly and draw $y_i \sim N(\mu_0(t_i), \Psi^{t_i}_{\theta_i, \sigma^2_i})$

In [7]:
t_sub = np.random.choice(t, size=30, replace=False)
t_sub = np.sort(t_sub)

